In [1]:
from apikey import apikey, API_KEY, BASE_URL, DEPLOYMENT_NAME, PINECONE_API_KEY, PINECONE_ENV
from langchain.chat_models import AzureChatOpenAI
import openai

In [2]:
import os
os.environ['OPENAI_API_KEY'] = apikey


In [4]:
import guidance

In [2]:


guidance.llm = guidance.llms.OpenAI(
    DEPLOYMENT_NAME,
    api_type = "azure",
    api_key=API_KEY,
    api_base=BASE_URL,
    api_version="2023-05-15",
    temperature=0)

program = guidance("""Tweak this proverb to apply to model instructions instead.
{{proverb}}
- {{book}} {{chapter}}:{{verse}}

UPDATE
Where there is no guidance{{gen 'rewrite' stop="\\n-"}}
- GPT {{#select 'chapter'}}9{{or}}10{{or}}11{{/select}}:{{gen 'verse'}}""")

executed_program = program(
    proverb = "Where there is no guidance, a people falls,\nbut in an abundance of counselors there is safety.",
    book = "Proverbs",
    chapter = 11,
    verse = 14
)

Stop program Tweak this proverb to apply to model instructions instead.
 {{proverb}} 
- {{book}} {{chapter}} : {{verse}} 

UPDATE
Where there is no guidance {{gen 'rewrite' stop="\n-"}} 
- GPT {{#select 'chapter'}} 9 {{or}} 10 {{or}} 11 {{/select}} : {{gen 'verse'}}

In [4]:
executed_program.variables()

{'llm': <guidance.llms._openai.OpenAI at 0x1125fe1d0>,
 'logging': False,
 'proverb': 'Where there is no guidance, a people falls,\nbut in an abundance of counselors there is safety.',
 'book': 'Proverbs',
 'chapter': 11,
 'verse': 14,
 '@raw_prefix': 'Tweak this proverb to apply to model instructions instead.\n{{!--GMARKER_START_variable_ref$&#123;&#123;proverb&#125;&#125;$--}}Where there is no guidance, a people falls,\nbut in an abundance of counselors there is safety.{{!--GMARKER_END_variable_ref$$--}}\n- {{!--GMARKER_START_variable_ref$&#123;&#123;book&#125;&#125;$--}}Proverbs{{!--GMARKER_END_variable_ref$$--}} {{!--GMARKER_START_variable_ref$&#123;&#123;chapter&#125;&#125;$--}}11{{!--GMARKER_END_variable_ref$$--}}:{{!--GMARKER_START_variable_ref$&#123;&#123;verse&#125;&#125;$--}}14{{!--GMARKER_END_variable_ref$$--}}\n\nUPDATE\nWhere there is no guidance{{!--GMARKER_START_gen$&#123;&#123;gen \'rewrite\' stop="\\n-"&#125;&#125;$--}}'}

In [3]:
executed_program["rewrite"]


KeyError: 'rewrite'

# Chat dialog

In [5]:
azure_llm = guidance.llms.OpenAI(
    DEPLOYMENT_NAME,
    api_type = "azure",
    api_key=API_KEY,
    api_base=BASE_URL,
    api_version="2023-05-15",
    temperature=0)

In [6]:

experts = guidance('''
{{#system~}}
You are a helpful and terse assistant.
{{~/system}}

{{#user~}}
I want a response to the following question:
{{query}}
Name 3 world-class experts (past or present) who would be great at answering this?
Don't answer the question yet.
{{~/user}}

{{#assistant~}}
{{gen 'expert_names' temperature=0 max_tokens=300}}
{{~/assistant}}

{{#user~}}
Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
{{~/user}}

{{#assistant~}}
{{gen 'answer' temperature=0 max_tokens=500}}
{{~/assistant}}
''', llm=azure_llm)


experts(query='How can I be more productive?')

Stop program 
 {{#system~}} 
You are a helpful and terse assistant.
 {{~/system}} 

 {{#user~}} 
I want a response to the following question:
 {{query}} 
Name 3 world-class experts (past or present) who would be great at answering this?
Don't answer the question yet.
 {{~/user}} 

 {{#assistant~}} 
 {{gen 'expert_names' temperature=0 max_tokens=300}} 
 {{~/assistant}} 

 {{#user~}} 
Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
 {{~/user}} 

 {{#assistant~}} 
 {{gen 'answer' temperature=0 max_tokens=500}} 
 {{~/assistant}}

In [10]:
gpt4_llm = guidance.llms.OpenAI("text-davinci-003")


In [11]:
document = """Modern chat-style models like ChatGPT and Alpaca are trained with special tokens that mark out "roles" for different areas of the prompt. Guidance supports these models through role tags that automatically map to the correct tokens or API calls for the current LLM. Below we show how a role-based guidance program enables simple multi-step reasoning and planning."""

summary = guidance(
        """
{{#system~}}
You are a world best summarizer. \n
Condense the text, capturing essential points and core ideas. Include relevant \
examples, omit excess details, and ensure the summary's length matches the \
original's complexity.
{{/system~}}
{{#user~}}
Summarize the following text:
---
{{document}}
{{/user~}}

{{#assistant~}}
{{gen 'summarization' temperature=0.2 max_tokens=100}}
{{/assistant~}}
""",
        llm=gpt4_llm,
    )  # pyright: ignore reportPrivateUsage=none

summary = summary(document=document)


Stop program 
 {{#system~}} 
You are a world best summarizer. 

Condense the text, capturing essential points and core ideas. Include relevant examples, omit excess details, and ensure the summary's length matches the original's complexity.
 {{/system~}} 
 {{#user~}} 
Summarize the following text:
---
 {{document}} 
 {{/user~}} 

 {{#assistant~}} 
 {{gen 'summarization' temperature=0.2 max_tokens=100}} 
 {{/assistant~}}

In [12]:
summary.summarization

AttributeError: 'Program' object has no attribute 'summarization'